In [1]:
import pandas as pd

##### Задача 1.  

Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [2]:
# Шаг 01. Подгружаем исходный датафрейм
ratings = pd.read_csv('ratings.csv')
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182


In [3]:
# Шаг 02. Через группировку рассчитываем кол-во выставленных оценок
ratings_grouped = ratings.groupby('userId').count().reset_index()
# Шаг 03. Отбираем только юзеров, выставивших более 100 оценок
ratings_filtered = ratings_grouped[ratings_grouped['rating']>=100]
# Шаг 04. Формируем список юзеров в лист
ratings_100_list = ratings_filtered['userId'].tolist()
# Шаг 05. Отбираем только тех юзеров, кто в списке. Т.е. поставивших более 100 оценок
ratings_base = ratings[ ratings['userId'].isin(ratings_100_list)]
ratings_base.head(3)

,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582


In [4]:
# Шаг 06.Функция разницы максимальной и минимальной даты
def groupby_function(ratings_base):
    return ratings_base.timestamp.max() - ratings_base.timestamp.min()

In [5]:
# Шаг 07. Выводим среднее время в днеях
print(f"Среднее время жизни составляет {round(ratings_base.groupby('userId')\
                                        .apply(groupby_function).mean()/86400,2)} дней")

Среднее время жизни составляет 455.23 дней


##### Задача 2.  
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [6]:
# Шаг 01. Загружаем все 4 датафрейма
rzd = pd.DataFrame(
    {'client_id': [111, 112, 113, 114, 115],
     'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [7]:
# Шаг 02. Объединяем все эти датафреймы в один
clients_df = client_base.merge(rzd,how='left').merge(auto,how='left').merge(air,how='left')
# Шаг 03. Выводим вторую часть задачи: с адресом
clients_df

,address,client_id,rzd_revenue,auto_revenue,air_revenue
0,Комсомольская 4,111,1093.0,NaN,NaN
1,Энтузиастов 8а,112,2810.0,NaN,NaN
2,Левобережная 1а,113,10283.0,57483.0,NaN
3,Мира 14,114,5774.0,83.0,NaN
4,ЗЖБИиДК 1,115,981.0,912.0,81.0
5,Строителей 18,116,NaN,4834.0,4.0
6,Панфиловская 33,117,NaN,98.0,13.0
7,Мастеркова 4,118,NaN,NaN,173.0


In [8]:
# Шаг 04. Выводим первую часть задачи: с адресом
clients_df[clients_df['rzd_revenue'].notnull() & clients_df['auto_revenue'].notnull()& clients_df['air_revenue'].notnull()]

,address,client_id,rzd_revenue,auto_revenue,air_revenue
4,ЗЖБИиДК 1,115,981.0,912.0,81.0


Задача 3.  
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

In [9]:
# Шаг 01. Загружаем все 2 датафрейма
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)
visits = visits[['user_id', 'source']]
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)
purchases = purchases[['user_id', 'category']]

In [10]:
# Шаг 02. Объединяем в один датафрейм, удаляем дубликаты
merge_df = visits.merge(purchases, how='outer',on='user_id')
merge_df.drop_duplicates(subset = ['user_id', 'source','category'], keep = 'first', inplace = True)
merge_df

,user_id,source,category
0,11,ad,Спорт
2,11,google,Спорт
4,22,yandex,Авто
5,55,email,Дача
6,77,ad,NaN
7,99,NaN,Авто


Следующие шаги:
Есть два варианта: 
1. Координаты известны только для **user_id**
2. Координаты известны для пары **user_id, source**  

Для обоих вариантов:
1. Удаление дубликатов(Вар.1:Сцепка по **user_id**, Вар.2:Сцепка по **user_id, source**)
2. Левая сцепка к **merge_df**
3. На всякий случай проверить ошибки и удалить дубликаты